Run this notebook to

- Merge all response epochs from all participants and all sessions and save merged epochs file
- Plot Pe for correct vs incorrect trials
- Plot Pe for correct vs incorrect trials split by partner condition
- Plot Pe for lower than median vs higher than median confidence 
- Plot Pe for lower than median vs higher than median confidence split by partner condition
- Plot all of the above separately for the strategic and observational conditions


## Import stuff

In [ ]:
import os
import numpy as np
import mne
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,corrmap)
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd

million = 1000000.


#%matplotlib qt
%matplotlib inline

input_dir = 'TaskresponseEpochsMastoids'


In [ ]:
def load_subj_eeg(path, file, downsample=None):
    fp = os.path.join(path, '%s-epo.fif' % file)
    print('>>> Loading %s' % fp)
    epochs = mne.read_epochs(fp, preload=True)
    if downsample is not None:
        epochs = epochs.resample(downsample)
    return epochs

def load_all_eeg(path, files, downsample=None):
    subject_epochs = [load_subj_eeg(path, file, downsample=downsample) for file in files]
    epochs = mne.epochs.concatenate_epochs(subject_epochs)
    return epochs

## Merge epochs

In [ ]:
participant_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21]

sessions = [1, 2]

participant_files = []

for session in sessions:
    for subject in participant_numbers:
        participant_files.append('%i_%i' % (subject, session))

In [ ]:
remerge = True
if remerge:
    response_epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    response_epochs.save('mergedData/response_epoch_mastoids-epo.fif', overwrite=True)
else:
    response_epochs = mne.read_epochs('mergedData/response_epoch_mastoids-epo.fif')
    
response_epochs = response_epochs.pick_types(eeg=True)
data = response_epochs.metadata

## Correct vs incorrect grand average (first calculating each participant's average)

In [ ]:
grand_average_correct = []
grand_average_incorrect = []

correct_epochs = response_epochs['participant_correct == True']
incorrect_epochs = response_epochs['participant_correct == False']

for subject in participant_numbers:
    grand_average_correct.append(correct_epochs['participant == %i' % subject].average())
    grand_average_incorrect.append(incorrect_epochs['participant == %i' % subject].average())


grand_average_correct = mne.grand_average(grand_average_correct)
grand_average_incorrect = mne.grand_average(grand_average_incorrect)

evokeds = dict(correct=grand_average_correct,
               incorrect=grand_average_incorrect)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


## Correct vs incorrect split by partner grand average 


In [ ]:
grand_average_underconf_correct = []
grand_average_underconf_incorrect = []
grand_average_overconf_correct = []
grand_average_overconf_incorrect = []

underconf_epochs = response_epochs['partner == "underconfident"']
underconf_correct_epochs = underconf_epochs['participant_correct == True']
underconf_incorrect_epochs = underconf_epochs['participant_correct == False']

overconf_epochs = response_epochs['partner == "overconfident"']
overconf_correct_epochs = overconf_epochs['participant_correct == True']
overconf_incorrect_epochs = overconf_epochs['participant_correct == False']



for subject in participant_numbers:    
    grand_average_underconf_correct.append(underconf_correct_epochs['participant == %i' % subject].average())
    grand_average_underconf_incorrect.append(underconf_incorrect_epochs['participant == %i' % subject].average())
    grand_average_overconf_correct.append(overconf_correct_epochs['participant == %i' % subject].average())
    grand_average_overconf_incorrect.append(overconf_incorrect_epochs['participant == %i' % subject].average())

grand_average_underconf_correct = mne.grand_average(grand_average_underconf_correct)
grand_average_underconf_incorrect = mne.grand_average(grand_average_underconf_incorrect)
grand_average_overconf_correct = mne.grand_average(grand_average_overconf_correct)
grand_average_overconf_incorrect = mne.grand_average(grand_average_overconf_incorrect)


evokeds = dict(underconf_incorrect=grand_average_underconf_incorrect,
               underconf_correct=grand_average_underconf_correct,
               overconf_incorrect=grand_average_overconf_incorrect,
               overconf_correct=grand_average_overconf_correct)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


## Low vs high confidence grand average 

In [ ]:
zscore = lambda x: (x - x.mean()) / x.std()
# zscore by participant
response_epochs.metadata['confidence_z_by_participant'] = response_epochs.metadata['participant_confidence'].groupby(response_epochs.metadata['participant']).transform(zscore)
        
median_confidence = response_epochs.metadata['confidence_z_by_participant'].median()


In [ ]:
grand_average_high_conf = []
grand_average_low_conf = []

high_conf_epochs = response_epochs['confidence_z_by_participant > 0']
low_conf_epochs = response_epochs['confidence_z_by_participant <= 0']


        
for subject in participant_numbers:
    grand_average_high_conf.append(high_conf_epochs['participant == %i' % subject].average())
    grand_average_low_conf.append(low_conf_epochs['participant == %i' % subject].average())


grand_average_high_conf = mne.grand_average(grand_average_high_conf)
grand_average_low_conf = mne.grand_average(grand_average_low_conf)

evokeds = dict(high_confidence=grand_average_high_conf,
               low_confidence=grand_average_low_conf)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


## Low vs high confidence split by partner grand average 


In [ ]:
grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_epochs = response_epochs['partner == "underconfident"']
underconf_high_conf_epochs = underconf_epochs['confidence_z_by_participant > 0']
underconf_low_conf_epochs = underconf_epochs['confidence_z_by_participant <= 0']

overconf_epochs = response_epochs['partner == "overconfident"']
overconf_high_conf_epochs = overconf_epochs['confidence_z_by_participant > 0']
overconf_low_conf_epochs = overconf_epochs['confidence_z_by_participant <= 0']



for subject in participant_numbers:    
    grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())

grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)


evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


In [ ]:
response_epochs_copy = response_epochs

# Strategic condition

In [ ]:
response_epochs = response_epochs_copy['condition == "s"']

## Correct vs incorrect grand average (first calculating each participant's average)

In [ ]:
grand_average_correct = []
grand_average_incorrect = []

correct_epochs = response_epochs['participant_correct == True']
incorrect_epochs = response_epochs['participant_correct == False']

for subject in participant_numbers:
    grand_average_correct.append(correct_epochs['participant == %i' % subject].average())
    grand_average_incorrect.append(incorrect_epochs['participant == %i' % subject].average())


grand_average_correct = mne.grand_average(grand_average_correct)
grand_average_incorrect = mne.grand_average(grand_average_incorrect)

evokeds = dict(correct=grand_average_correct,
               incorrect=grand_average_incorrect)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


## Correct vs incorrect split by partner grand average 


In [ ]:
grand_average_underconf_correct = []
grand_average_underconf_incorrect = []
grand_average_overconf_correct = []
grand_average_overconf_incorrect = []

underconf_epochs = response_epochs['partner == "underconfident"']
underconf_correct_epochs = underconf_epochs['participant_correct == True']
underconf_incorrect_epochs = underconf_epochs['participant_correct == False']

overconf_epochs = response_epochs['partner == "overconfident"']
overconf_correct_epochs = overconf_epochs['participant_correct == True']
overconf_incorrect_epochs = overconf_epochs['participant_correct == False']



for subject in participant_numbers:    
    grand_average_underconf_correct.append(underconf_correct_epochs['participant == %i' % subject].average())
    grand_average_underconf_incorrect.append(underconf_incorrect_epochs['participant == %i' % subject].average())
    grand_average_overconf_correct.append(overconf_correct_epochs['participant == %i' % subject].average())
    grand_average_overconf_incorrect.append(overconf_incorrect_epochs['participant == %i' % subject].average())

grand_average_underconf_correct = mne.grand_average(grand_average_underconf_correct)
grand_average_underconf_incorrect = mne.grand_average(grand_average_underconf_incorrect)
grand_average_overconf_correct = mne.grand_average(grand_average_overconf_correct)
grand_average_overconf_incorrect = mne.grand_average(grand_average_overconf_incorrect)


evokeds = dict(underconf_incorrect=grand_average_underconf_incorrect,
               underconf_correct=grand_average_underconf_correct,
               overconf_incorrect=grand_average_overconf_incorrect,
               overconf_correct=grand_average_overconf_correct)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


## Low vs high confidence grand average 

In [ ]:
zscore = lambda x: (x - x.mean()) / x.std()
# zscore by participant
response_epochs.metadata['confidence_z_by_participant'] = response_epochs.metadata['participant_confidence'].groupby(response_epochs.metadata['participant']).transform(zscore)
        
median_confidence = response_epochs.metadata['confidence_z_by_participant'].median()

# # then zscore by partner (check if this is okay to do like this!!!)
# response_epochs.metadata['confidence_z_by_participant_by_partner'] = response_epochs.metadata['confidence_z_by_participant'].groupby(response_epochs.metadata['partner']).transform(zscore)


In [ ]:
grand_average_high_conf = []
grand_average_low_conf = []

high_conf_epochs = response_epochs['confidence_z_by_participant > 0']
low_conf_epochs = response_epochs['confidence_z_by_participant <= 0']


        
for subject in participant_numbers:
    grand_average_high_conf.append(high_conf_epochs['participant == %i' % subject].average())
    grand_average_low_conf.append(low_conf_epochs['participant == %i' % subject].average())


grand_average_high_conf = mne.grand_average(grand_average_high_conf)
grand_average_low_conf = mne.grand_average(grand_average_low_conf)

evokeds = dict(high_confidence=grand_average_high_conf,
               low_confidence=grand_average_low_conf)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


## Low vs high confidence split by partner grand average 


In [ ]:
grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_epochs = response_epochs['partner == "underconfident"']
underconf_high_conf_epochs = underconf_epochs['confidence_z_by_participant > 0']
underconf_low_conf_epochs = underconf_epochs['confidence_z_by_participant <= 0']

overconf_epochs = response_epochs['partner == "overconfident"']
overconf_high_conf_epochs = overconf_epochs['confidence_z_by_participant > 0']
overconf_low_conf_epochs = overconf_epochs['confidence_z_by_participant <= 0']



for subject in participant_numbers:    
    grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())

grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)


evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


# Observational condition

In [ ]:
response_epochs = response_epochs_copy['condition == "ns"']

## Correct vs incorrect grand average (first calculating each participant's average)

In [ ]:
grand_average_correct = []
grand_average_incorrect = []

correct_epochs = response_epochs['participant_correct == True']
incorrect_epochs = response_epochs['participant_correct == False']

for subject in participant_numbers:
    grand_average_correct.append(correct_epochs['participant == %i' % subject].average())
    grand_average_incorrect.append(incorrect_epochs['participant == %i' % subject].average())


grand_average_correct = mne.grand_average(grand_average_correct)
grand_average_incorrect = mne.grand_average(grand_average_incorrect)

evokeds = dict(correct=grand_average_correct,
               incorrect=grand_average_incorrect)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


## Correct vs incorrect split by partner grand average 


In [ ]:
grand_average_underconf_correct = []
grand_average_underconf_incorrect = []
grand_average_overconf_correct = []
grand_average_overconf_incorrect = []

underconf_epochs = response_epochs['partner == "underconfident"']
underconf_correct_epochs = underconf_epochs['participant_correct == True']
underconf_incorrect_epochs = underconf_epochs['participant_correct == False']

overconf_epochs = response_epochs['partner == "overconfident"']
overconf_correct_epochs = overconf_epochs['participant_correct == True']
overconf_incorrect_epochs = overconf_epochs['participant_correct == False']



for subject in participant_numbers:    
    grand_average_underconf_correct.append(underconf_correct_epochs['participant == %i' % subject].average())
    grand_average_underconf_incorrect.append(underconf_incorrect_epochs['participant == %i' % subject].average())
    grand_average_overconf_correct.append(overconf_correct_epochs['participant == %i' % subject].average())
    grand_average_overconf_incorrect.append(overconf_incorrect_epochs['participant == %i' % subject].average())

grand_average_underconf_correct = mne.grand_average(grand_average_underconf_correct)
grand_average_underconf_incorrect = mne.grand_average(grand_average_underconf_incorrect)
grand_average_overconf_correct = mne.grand_average(grand_average_overconf_correct)
grand_average_overconf_incorrect = mne.grand_average(grand_average_overconf_incorrect)


evokeds = dict(underconf_incorrect=grand_average_underconf_incorrect,
               underconf_correct=grand_average_underconf_correct,
               overconf_incorrect=grand_average_overconf_incorrect,
               overconf_correct=grand_average_overconf_correct)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


## Low vs high confidence grand average 

In [ ]:
zscore = lambda x: (x - x.mean()) / x.std()
# zscore by participant
response_epochs.metadata['confidence_z_by_participant'] = response_epochs.metadata['participant_confidence'].groupby(response_epochs.metadata['participant']).transform(zscore)
        
median_confidence = response_epochs.metadata['confidence_z_by_participant'].median()

# # then zscore by partner (check if this is okay to do like this!!!)
# response_epochs.metadata['confidence_z_by_participant_by_partner'] = response_epochs.metadata['confidence_z_by_participant'].groupby(response_epochs.metadata['partner']).transform(zscore)


In [ ]:
grand_average_high_conf = []
grand_average_low_conf = []

high_conf_epochs = response_epochs['confidence_z_by_participant > 0']
low_conf_epochs = response_epochs['confidence_z_by_participant <= 0']


        
for subject in participant_numbers:
    grand_average_high_conf.append(high_conf_epochs['participant == %i' % subject].average())
    grand_average_low_conf.append(low_conf_epochs['participant == %i' % subject].average())


grand_average_high_conf = mne.grand_average(grand_average_high_conf)
grand_average_low_conf = mne.grand_average(grand_average_low_conf)

evokeds = dict(high_confidence=grand_average_high_conf,
               low_confidence=grand_average_low_conf)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)


## Low vs high confidence split by partner grand average 


In [ ]:
grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_epochs = response_epochs['partner == "underconfident"']
underconf_high_conf_epochs = underconf_epochs['confidence_z_by_participant > 0']
underconf_low_conf_epochs = underconf_epochs['confidence_z_by_participant <= 0']

overconf_epochs = response_epochs['partner == "overconfident"']
overconf_high_conf_epochs = overconf_epochs['confidence_z_by_participant > 0']
overconf_low_conf_epochs = overconf_epochs['confidence_z_by_participant <= 0']



for subject in participant_numbers:    
    grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())

grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)


evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)
